# Import Module:

In [1]:
import pandas as pd #Untuk dataframe manipulation
import numpy as np #Untuk numerical python array
import matplotlib.pyplot as plt #Untuk visualisasi data
import seaborn as sns #Untuk visualisasi data
from pandas.tseries.offsets import BDay #Settings pandas

#Dataset

Data yang digunakan adalah data yang berasal dari Tokopedia (***bukan data sesungguhnya***). Mengenai penjelasan dataset adalah sebagai berikut:

|variable                       |class     |description |
|:------------------------------|:---------|:-----------|
**order_detail:**
id 			|object| angka unik dari order / id_order
customer_id 		|object|angka unik dari pelanggan
order_date 		|object| tanggal saat dilakukan transaksi
sku_id 			|object| angka unik dari produk (sku adalah stock keeping unit)
price			|int64| harga yang tertera pada tagging harga
qty_ordered 		|int64| jumlah barang yang dibeli oleh pelanggan
before_discount	|float64| nilai harga total dari produk (price * qty_ordered)
discount_amount	|float64| nilai diskon product total
after_discount		|float64| nilai harga total produk ketika sudah dikurangi dengan diskon
is_gross 		|int64| menunjukkan pelanggan belum membayar pesanan
is_valid		|int64| menunjukkan pelanggan sudah melakukan pembayaran
is_net			|int64| menunjukkan transaksi sudah selesai
payment_id 		|int64| angka unik dari metode pembayaran
||
**sku_detail:**
id |object| angka unik dari produk (dapat digunakan untuk key saat join)
sku_name 		|object| nama dari produk
base_price		|float64| harga barang yang tertera pada tagging harga / price
cogs 			|int64| cost of goods sold / total biaya untuk menjual 1 produk
category		|object| kategori produk
||
**customer_detail:**
id 			|object| angka unik dari pelanggan
registered_date	|object| tanggal pelanggan mulai mendaftarkan diri sebagai anggota
||
**payment_detail:**
id			|int64| angka unik dari metode pembayaran
payment_method	|object| metode pembayaran yang digunakan


# Tipe File Data
1. Comma Seperated Value (CSV)
2. Java Script Object Notation (JSON)
3. Hadoop Distirbuted FIle System (HDF)

In [2]:
#Sumber data yang digunakan
path_od = "https://raw.githubusercontent.com/dataskillsboost/FinalProjectDA11/main/order_detail.csv"
path_pd = "https://raw.githubusercontent.com/dataskillsboost/FinalProjectDA11/main/payment_detail.csv"
path_cd = "https://raw.githubusercontent.com/dataskillsboost/FinalProjectDA11/main/customer_detail.csv"
path_sd = "https://raw.githubusercontent.com/dataskillsboost/FinalProjectDA11/main/sku_detail.csv"
df_od = pd.read_csv(path_od)
df_pd = pd.read_csv(path_pd)
df_cd = pd.read_csv(path_cd)
df_sd = pd.read_csv(path_sd)

In [3]:
#Mengampilkan 5 baris pertama
df_od.head()

,id,customer_id,order_date,sku_id,price,qty_ordered,before_discount,discount_amount,after_discount,is_gross,is_valid,is_net,payment_id
0,ODR9939707760w,C713589L,2021-11-19,P858068,26100,200,5220000.0,2610000.00,2610000.00,1,1,0,5
1,ODR7448356649d,C551551L,2021-11-19,P886455,1971942,5,9859710.0,2464927.50,7394782.50,1,0,0,5
2,ODR4011281866z,C685596L,2021-11-25,P678648,7482000,1,7482000.0,2065344.62,5416655.38,1,0,0,4
3,ODR3378927994s,C830683L,2021-11-22,P540013,3593680,1,3593680.0,1455440.40,2138239.60,1,1,1,5
4,ODR4904430099k,C191766L,2021-11-21,P491032,4413220,1,4413220.0,1059172.80,3354047.20,1,1,1,4


In [4]:
#Mengampilkan 5 baris terakhir
df_pd.tail()

,id,payment_method
11,12,internetbanking
12,13,Easypay_MA
13,14,productcredit
14,15,marketingexpense
15,16,financesettlement


In [5]:
#Mengampilkan 5 baris acak
df_cd.sample(n = 5)

,id,registered_date
1411,C955839L,2021-10-06
887,C160222L,2021-08-11
569,C405083L,2021-08-16
2103,C191188L,2021-12-08
951,C508126L,2021-09-09


In [6]:
#Mengampilkan 10% baris acak
df_cd.sample(frac = 0.05)

,id,registered_date
250,C189483L,2021-07-22
707,C495724L,2021-08-22
2403,C597119L,2022-01-02
2459,C929625L,2022-01-14
3330,C690725L,2022-05-04
...,...,...
102,C948868L,2021-07-27
3837,C149895L,2022-07-20
3796,C238230L,2022-07-08
2339,C114766L,2022-01-28


In [7]:
#Mengampilkan 3 baris pertama
df_sd.head(3)

,id,sku_name,base_price,cogs,category
0,P798444,AT-FSM-35,57631.70,46052,Kids & Baby
1,P938347,AYS_Haier-18HNF,3931789.26,3499256,Appliances
2,P826364,Atalian _DV206A-Brown-41,324597.00,243426,Men Fashion


In [10]:
#Menjalankan SQL di Colab
from sqlite3 import connect
conn = connect(':memory:')
df_od.to_sql('order_detail',conn, index=False, if_exists='replace')
df_pd.to_sql('payment_detail', conn, index=False, if_exists='replace')
df_sd.to_sql('sku_detail', conn, index=False, if_exists='replace')
df_cd.to_sql('customer_detail', conn, index=False, if_exists='replace')

3998

In [9]:
#Query SQL untuk menggabungkan data
df = pd.read_sql("""
SELECT
    order_detail.*,
    payment_detail.payment_method,
    sku_detail.sku_name,
    sku_detail.base_price,
    sku_detail.cogs,
    sku_detail.category,
    customer_detail.registered_date
FROM order_detail
LEFT JOIN payment_detail
    on payment_detail.id = order_detail.payment_id
LEFT JOIN sku_detail
    on sku_detail.id = order_detail.sku_id
LEFT JOIN customer_detail
    on customer_detail.id = order_detail.customer_id
""", conn)

In [11]:
#Mengampilkan 5 baris pertama
df.head()

,id,customer_id,order_date,sku_id,price,qty_ordered,before_discount,discount_amount,after_discount,is_gross,is_valid,is_net,payment_id,payment_method,sku_name,base_price,cogs,category,registered_date
0,ODR9939707760w,C713589L,2021-11-19,P858068,26100,200,5220000.0,2610000.00,2610000.00,1,1,0,5,jazzwallet,RB_Dettol Germ Busting Kit-bf,26100.0,18270,Others,2021-07-07
1,ODR7448356649d,C551551L,2021-11-19,P886455,1971942,5,9859710.0,2464927.50,7394782.50,1,0,0,5,jazzwallet,PS4_Slim-500GB,1971942.0,1321182,Entertainment,2021-11-20
2,ODR4011281866z,C685596L,2021-11-25,P678648,7482000,1,7482000.0,2065344.62,5416655.38,1,0,0,4,Payaxis,Changhong Ruba 55 Inches UD55D6000i Ultra HD T...,7482000.0,5162580,Entertainment,2021-11-19
3,ODR3378927994s,C830683L,2021-11-22,P540013,3593680,1,3593680.0,1455440.40,2138239.60,1,1,1,5,jazzwallet,dawlance_Inverter 30,3593680.0,3054628,Appliances,2021-11-03
4,ODR4904430099k,C191766L,2021-11-21,P491032,4413220,1,4413220.0,1059172.80,3354047.20,1,1,1,4,Payaxis,Dawlance_Inverter-45 2.0 ton,4413220.0,3177472,Appliances,2021-07-05


In [12]:
#Menampikan tipe data tiap kolom
df.dtypes

id                  object
customer_id         object
order_date          object
sku_id              object
price                int64
qty_ordered          int64
before_discount    float64
discount_amount    float64
after_discount     float64
is_gross             int64
is_valid             int64
is_net               int64
payment_id           int64
payment_method      object
sku_name            object
base_price         float64
cogs                 int64
category            object
registered_date     object
dtype: object

In [13]:
#Mengubah tipe data agar mudah dilakukan pengolahan data
df = df.astype({"before_discount":'int', "discount_amount":'int', "after_discount":'int',"base_price":'int'})
df.dtypes

id                 object
customer_id        object
order_date         object
sku_id             object
price               int64
qty_ordered         int64
before_discount     int64
discount_amount     int64
after_discount      int64
is_gross            int64
is_valid            int64
is_net              int64
payment_id          int64
payment_method     object
sku_name           object
base_price          int64
cogs                int64
category           object
registered_date    object
dtype: object

In [14]:
#Mengubah tipe kolom Date menjadi Datetime
df['order_date']= pd.to_datetime(df['order_date'])
df['registered_date']= pd.to_datetime(df['registered_date'])
df.dtypes


id                         object
customer_id                object
order_date         datetime64[ns]
sku_id                     object
price                       int64
qty_ordered                 int64
before_discount             int64
discount_amount             int64
after_discount              int64
is_gross                    int64
is_valid                    int64
is_net                      int64
payment_id                  int64
payment_method             object
sku_name                   object
base_price                  int64
cogs                        int64
category                   object
registered_date    datetime64[ns]
dtype: object

## `No 1`
**Dear Data Analyst**,
<br></br>
Akhir tahun ini, perusahaan akan memberikan hadiah bagi pelanggan yang memenangkan kompetisi **Festival Akhir Tahun**. Tim Marketing membutuhkan bantuan untuk menentukan perkiraan hadiah yang akan diberikan pada pemenang kompetisi nantinya. Hadiah tersebut akan diambil dari **TOP 5 Produk** dari Kategori **Mobiles & Tablets** selama tahun 2022, dengan jumlah kuantitas penjualan (valid = 1) paling tinggi.

Mohon bantuan, untuk mengirimkan data tersebut sebelum akhir bulan ini ke Tim Marketing. Atas bantuan yang diberikan, kami mengucapkan terima kasih.
<br></br>
Regards

**Tim Marketing**

<details><summary>Klik di sini untuk tips</summary>


```python
1. Memfilter data dengan valid = 1
2. Memfilter data dengan Category = Mobiles & Tablets
3. Memfilter data pada transaksi selama 2022
4. Gunakan groupby berdasarkan sku_name
5. Gunakan sort_values untuk mengurutkan data
6. Gunakan head untuk menampilkan top 5 produk


```



</details>

## `Jawaban No 1`

In [55]:
# Tulis kode Anda di bawah ini. Dapat menggunakan lebih dari 1 blok kode
df["category"].unique()

array(['Others', 'Entertainment', 'Appliances', 'Computing',
       'Mobiles & Tablets', 'Superstore', 'Health & Sports',
       'Women Fashion', 'Home & Living', 'Men Fashion',
       'Beauty & Grooming', 'Soghaat', 'School & Education', 'Books',
       'Kids & Baby'], dtype=object)

In [56]:
mt_category = 'Mobiles & Tablets'

In [57]:
df["order_date"].dt.year

0       2021
1       2021
2       2021
3       2021
4       2021
        ... 
5879    2022
5880    2022
5881    2022
5882    2022
5883    2022
Name: order_date, Length: 5884, dtype: int32

In [58]:
#Cara filter di pandas
filter_criteria = (df["is_valid"] == 1) &\
 (df["category"] == mt_category)  &\
  (df["order_date"] >= '2022-01-01') &\
   (df["order_date"] <= '2022-12-31')
 #(df["order_date"].dt.year == 2022)

In [59]:

filter_criteria = (df["is_valid"] == 1) &\
 (df["category"] == 'Mobiles & Tablets')  &\
  (df["order_date"] >= '2022-01-01') &\
   (df["order_date"] <= '2022-01-31')

In [60]:
df_filtered = df[(df["is_valid"] == 1) &\
 (df["category"] == 'Mobiles & Tablets')  &\
  (df["order_date"].dt.year == 2022)]

### Notes:
**Filter**
- AND: &
- OR: |
- NOT: ~ atau NOT()

**Operation**
- ==: sama dengan
- \>, >= : lebih besar dan lbh bsr sm dgn
- <. <=: lebih kecil dan lbh kcl sm dgn
- !=: Tidak sama dengan

In [22]:
sku_qty = df_filtered.groupby(["sku_name"]).agg({"qty_ordered" : "sum",
                                                 "after_discount" : "sum"})
sku_qty.sort_values("qty_ordered", ascending = False).head()

,qty_ordered,after_discount
sku_name,,
IDROID_BALRX7-Gold,1000,518752000
IDROID_BALRX7-Jet black,31,17079202
Infinix Hot 4-Gold,15,10787130
samsung_Grand Prime Plus-Black,11,10207362
infinix_Zero 4-Grey,10,13339420


In [23]:
top_5_sku = sku_qty.sort_values("qty_ordered", ascending = False).head().reset_index()

In [24]:
top_5_sku.to_csv("report top 5 procut.csv", index = False)

In [25]:
sku_qty1 = df_filtered.groupby(["sku_name"]).agg({"qty_ordered" : ["sum", "mean"]})
sku_qty1.sort_values(("qty_ordered", "sum"), ascending = False)

qty_ordered             
                                                  sum         mean
sku_name                                                          
IDROID_BALRX7-Gold                               1000  1000.000000
IDROID_BALRX7-Jet black                            31    15.500000
Infinix Hot 4-Gold                                 15     7.500000
samsung_Grand Prime Plus-Black                     11    11.000000
infinix_Zero 4-Grey                                10     3.333333
samsung_Grand Prime Plus-Silver                    10    10.000000
iphone_7-128GB-wof-Matt Black                       6     1.000000
Samsung-Galaxy-S8-G955-Plus-Black                   5     1.250000
iphone_7-128GB-wof-Jet Black                        5     1.000000
samsungGALAXY J510F-2016 DS LTE-Gold                4     4.000000
cc_samsung_G935F-Blue                               4     2.000000
Apple iPhone 6S Plus 16GB Silver                    3     1.000000
cc_Samsung_Galaxy_S8_Plus_Black                     3     1.000000
Samsung_S7 Edge-Gold                                3     1.000000
Samsung_Galaxy_S8_black                             2     1.000000
huawei_Mate-9-Black                                 2     1.000000
iPhone7-Red-128GB                                   2     1.000000
cc_Samsung_Galaxy_S8_Plus_Gold                      2     1.000000
cc_Samsung_Galaxy_S8_black                          2     1.000000
iPhone7Plus-Red-256GB                               2     1.000000
oppo_f3-plus-Gold                                   2     2.000000
iPhone7- Jet Black-128GB                            2     1.000000
cc_Huawei-Mate-9-Grey                               2     1.000000
Samsung_A7-2017-Gold                                2     2.000000
iphone-7-32gb-wof-Matt Black                        2     1.000000
samsung_A720-Black                                  1     1.000000
samsung_Samsung Galaxy J7 PRIME 2016-Gold           1     1.000000
samsung_G935F-Gold                                  1     1.000000
samsung_G935F-Black                                 1     1.000000
iPhone7Plus-Jet Black-256GB                         1     1.000000
iPhone7Plus-Matte Black-128GB                       1     1.000000
huawei_Google Nexus 6P-Silver                       1     1.000000
iPhone7-Red-256GB                                   1     1.000000
iPhone7-Matte Black -32GB                           1     1.000000
huawei_Mate 9-mocjha brown                          1     1.000000
huawei_Mate 9-gold                                  1     1.000000
cc_samsung_a320-Gold                                1     1.000000
cc_samsung_Grand Prime Plus-Silver                  1     1.000000
cc_samsung_A520-Gold                                1     1.000000
cc_Samsung_Galaxy_S8_gold                           1     1.000000
cc_Huawei-Mate-9-Brown                              1     1.000000
cc-hTC-One-A9-silver                                1     1.000000
Samsung_S7 Edge-Black                               1     1.000000
Samsung_Galaxy_S8_Plus_Black                        1     1.000000
MotoZPlay-White                                     1     1.000000
MotoZPlay -Black                                    1     1.000000
samsung_galaxy-s7-edge-pearl-black                  1     1.000000

## `No 2`
**Dear Data Analyst**,
<br></br>
Menindaklanjuti meeting gabungan Tim Werehouse dan Tim Marketing, kami menemukan bahwa ketersediaan stock produk dengan Kategori Others pada akhir 2022 kemarin masih banyak.
1. Kami mohon bantuan untuk melakukan pengecekan data penjualan kategori tersebut dengan tahun 2021 secara kuantitas penjualan. Dugaan sementara kami, telah terjadi penurunan kuantitas penjualan pada 2022 dibandingkan 2021. (Mohon juga menampilkan data ke-15 kategori)
<br></br>
2. Apabila memang terjadi penurunan kuantitas penjualan pada kategori Others, kami mohon bantuan untuk menyediakan data TOP 20 nama produk yang mengalami penurunan paling tinggi pada 2022 jika dibanding dengan 2021. Hal ini kami gunakan sebagai bahan diskusi pada meeting selanjutnya.  

Mohon bantuan untuk mengirimkan data tersebut paling lambat 4 hari dari hari ini. Atas bantuan yang diberikan, kami mengucapkan terima kasih.
<br></br>
Regards

**Tim Werehouse**

<details><summary>Klik di sini untuk tips</summary>


```python
1. Memfilter data dengan valid = 1
2. Memfilter data pada transaksi selama 2021
3. Memfilter data pada transaksi selama 2022
4. Gunakan groupby berdasarkan sku_name masing-masing tahun
5. Menggabungkan kedua data dengan merge
6. Lakukan pengurangan kolom qty 2022 dengan qty 2021


```



</details>

## `Jawaban No 2.1`

In [26]:
# Tulis kode Anda di bawah ini. Dapat menggunakan lebih dari 1 blok kode
cat_2021 = df[
    (df["is_valid"] == 1) &\
  (df["order_date"].dt.year == 2021)
  ]\
  .groupby(["category"], as_index = False)["qty_ordered"].sum()
cat_2021

,category,qty_ordered
0,Appliances,124
1,Beauty & Grooming,168
2,Books,171
3,Computing,109
4,Entertainment,77
5,Health & Sports,173
6,Home & Living,193
7,Kids & Baby,170
8,Men Fashion,237
9,Mobiles & Tablets,107


In [28]:
cat_2022 = df[(df["is_valid"] == 1) &\
  (df["order_date"].dt.year == 2022)]\
  .groupby(["category"], as_index = False)["qty_ordered"].sum()
cat_2022

,category,qty_ordered
0,Appliances,148
1,Beauty & Grooming,153
2,Books,195
3,Computing,153
4,Entertainment,150
5,Health & Sports,200
6,Home & Living,250
7,Kids & Baby,227
8,Men Fashion,175
9,Mobiles & Tablets,1154


In [29]:
cat_performance = cat_2021.merge(cat_2022,
               left_on = "category",
               right_on = "category",
               how = "inner",
               suffixes = ("_2021", "_2022"))

cat_performance["selisih penjualan"] = cat_performance["qty_ordered_2022"] - cat_performance["qty_ordered_2021"]

In [30]:
cat_performance.sort_values("selisih penjualan")

,category,qty_ordered_2021,qty_ordered_2022,selisih penjualan
10,Others,426,263,-163
12,Soghaat,759,612,-147
8,Men Fashion,237,175,-62
1,Beauty & Grooming,168,153,-15
0,Appliances,124,148,24
2,Books,171,195,24
5,Health & Sports,173,200,27
3,Computing,109,153,44
11,School & Education,184,237,53
6,Home & Living,193,250,57


## `Jawaban No 2.2`

In [43]:
# Tulis kode Anda di bawah ini. Dapat menggunakan lebih dari 1 blok kode
others_2022 = df[(df["is_valid"] == 1) &\
 (df["category"] == 'Others')  &\
  (df["order_date"].dt.year == 2022)]\
  .groupby(["sku_name"], as_index = False)["qty_ordered"].sum()

others_2021 = df[(df["is_valid"] == 1) &\
 (df["category"] == 'Others')  &\
  (df["order_date"].dt.year == 2021)]\
  .groupby(["sku_name"], as_index = False)["qty_ordered"].sum()

others_performance = others_2021.merge(others_2022,
               left_on = "sku_name",
               right_on = "sku_name",
               how = "inner",
               suffixes = ("_2021", "_2022"))

others_performance["selisih penjualan"] = others_performance["qty_ordered_2022"] - others_performance["qty_ordered_2021"]

In [44]:
others_performance.sort_values("selisih penjualan").head(20)

,sku_name,qty_ordered_2021,qty_ordered_2022,selisih penjualan
7,RB_Dettol Germ Busting Kit-bf,200,45,-155
8,Telemall_MM-DR-HB-L,23,2,-21
16,kansai_NeverWet,10,1,-9
14,emart_00-1,7,1,-6
5,MEGUIAR_G12711,4,1,-3
0,Aladdin_bike_cover,3,2,-1
11,aw_Ultra Shine Wash & Wax-64oz./1893ml,2,1,-1
9,Tmall_MM-DR-PAD,2,1,-1
2,Entertainer Asia_Vouch 365-2017 Mobile App Lahore,2,1,-1
4,MEGUIAR_A1214,1,1,0


## `No 3`
**Dear Data Analyst**,
<br></br>
Terkait ulang tahun perusahaan pada 2 bulan mendatang, Tim Digital Marketing akan memberikan informasi promo bagi pelanggan pada akhir bulan ini. Kriteria pelanggan yang akan kami butuhkan adalah mereka yang sudah melakukan check-out namun belum melakukan pembayaran (is_gross = 1) selama tahun 2022. Data yang kami butuhkan adalah ID Customer dan Registered Date.

Mohon bantuan, untuk mengirimkan data tersebut sebelum akhir bulan ini ke Tim Digital Marketing. Atas bantuan yang diberikan, kami mengucapkan terima kasih.
<br></br>
Regards

**Tim Digital Marketing**

<details><summary>Klik di sini untuk tips</summary>


```python
1. Memfilter data dengan gross = 1
2. Memfilter data dengan valid = 0
3. Memfilter data dengan net = 0
4. Memfilter data pada transaksi selama 2022


```



</details>

## `Jawaban No 3`

In [45]:
customer_gross = df.loc[(df["is_gross"] == 1) &\
 (df["is_valid"] == 0) &\
  (df["is_net"] == 0) &\
   (df["order_date"].dt.year == 2022), ["customer_id", "registered_date"]]

In [47]:
customer_gross = customer_gross.drop_duplicates()
customer_gross

,customer_id,registered_date
9,C246762L,2022-05-08
18,C848774L,2021-11-07
19,C693415L,2022-04-12
21,C180595L,2022-04-22
22,C587425L,2022-03-22
...,...,...
5855,C653797L,2022-04-03
5856,C394076L,2021-10-12
5859,C248585L,2022-07-10
5865,C471304L,2022-05-13


In [53]:
#Jalankan kode ini untuk mendownload file
from google.colab import files
[customer_gross file].to_csv('audience_list.csv', encoding = 'utf-8-sig',index=False) #ganti [nama variabel file] dengan nama variabel yang digunakan
files.download('audience_list.csv')

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-53-831319a27cc9>, line 3)

## `No 4`
**Dear Data Analyst**,
<br></br>
Pada bulan October hingga Desember 2022, kami melakukan campaign setiap hari Sabtu dan Minggu. Kami hendak menilai, apakah campaign tersebut cukup berdampak pada kenaikan penjualan (before_discount). Mohon bantuan untuk menampilkan data:
1. Rata-rata harian penjualan weekends (Sabtu dan Minggu) vs rata-rata harian penjualan weekdays (Senin-Jumat) per bulan tersebut. Apakah ada peningkatan penjualan pada masing-masing bulan tersebut.
2. Rata-rata harian penjualan weekends (Sabtu dan Minggu) vs rata-rata harian penjualan weekdays (Senin-Jumat) keseluruhan 3 bulan tersebut.

Mohon bantuan untuk mengirimkan data tersebut paling lambat minggu depan. Atas bantuan yang diberikan, kami mengucapkan terima kasih.
<br></br>
Regards

**Tim Campaign**

<details><summary>Klik di sini untuk tips</summary>


```python
1. Buatlah kolom baru untuk day, month, dan month number
2. Memfilter data dengan valid = 1
3. Memfilter data dengan day termasuk 'Saturday', 'Sunday' dengan menggunakan fungsi isin dan simpan dalam variable 'weekends'
4. Memfilter data dengan day termasuk 'Monday','Tuesday','Wednesday','Thusday','Friday' dengan menggunakan fungsi isin dan simpan dengan variabel 'weekday'
5. Memfilter data pada transaksi selama 2022 bulan Oktober sampai dengan Desember
6. Gunakan groupby berdasarkan pada masing-masing variabel
7. Menggabungkan kedua data dengan merge
8. Tampilkan pada grafik batang untuk melihat perbedaananya


```



</details>

## `Jawaban No 4.1`

In [49]:
# Tulis kode Anda di bawah ini. Dapat menggunakan lebih dari 1 blok kode
df["is_weekend"] = df["order_date"].dt.weekday > 4
df["month_name"] = df["order_date"].dt.month_name()

In [50]:
df_filtered = df[(df["order_date"].dt.month >= 10) &\
                 (df["order_date"].dt.year == 2022) &\
                 (df["is_valid"] == 1)
                 ]

In [51]:
q4 = df_filtered.groupby(["month_name","is_weekend"], as_index = False)["before_discount"].mean()


## `Jawaban No 4.2`

In [52]:
# Tulis kode Anda di bawah ini. Dapat menggunakan lebih dari 1 blok kode
q4.groupby("is_weekend")["before_discount"].mean()

is_weekend
False    776708.850794
True     550884.561533
Name: before_discount, dtype: float64